In [17]:

import pandas as pd
import arff
import os 
import re

In [2]:
# Import metadata 
dimension=pd.read_csv('dimension/dimension.csv',delimiter=';')
sector=pd.read_csv('dimension/sector_dimension.csv')

In [50]:
# Clean the .arff file:
# Step 1: Clean random letters appearing and convert them to m
# Step 2: Clean the letter m in the Sector column which should be numeric,
# we know it`s the last column, convert m to ?
# which is the null value for numeric attributes in .arff files
# Step 3: Remove rows where the country Italy appears inb the first column


for filename in os.listdir('data/'):# Loop through each file in the directory
    with open(f'data/{filename}', 'r') as rf: # Open the .arff file
        # Read lines from the file
        lines = rf.readlines()

    for i, line in enumerate(lines):
        
        # Step 1: 
        # Clean random letters appearing and convert them to m
        pattern = r',([a-z]),'# Regex to match comma + single lowercase letter + comma
        # Perform the substitution
        lines[i] = re.sub(pattern, lambda x: ',m,', line)
        
        # Step 2:
        # If there is an m in the last three characters
        if 'm' in line[-3:]:
            # If yes, replace 'm' with '?' in the last 3 characters
            lines[i] = line[:-3] + line[-3:].replace('m', '?')
    
        # Step 3:
        # Split the string by commas
        string_elements = line.split(',')
        # Get the first value
        first_value = string_elements[0]

        # Check if the first value is Italy
        if first_value == 'Italy':
            # If not, mark the line with 'Remove'
            lines[i] = 'Remove'
            
    # Let`s remove all the lines=='Remove'
    lines = [line for line in lines if line != 'Remove']     
    
    # Write modified lines back to a new ARFF file
    with open(f'data_modified/{filename}', 'w') as wf:
        wf.writelines(lines)

In [51]:
# Import financial data

# Initialize an empty list to store DataFrames
dfs_list = []

for filename in os.listdir('data_modified/'):# Loop through each file in the directory
    with open(f'data_modified/{filename}', 'r') as f: # Open the .arff file
    
        raw_data = arff.load(f) # dict obj
        # Store arff in a pandas df
        df = pd.DataFrame(raw_data['data'],columns=[x[0] for x in raw_data['attributes']])
        
        # Create a dict with the mappings
        mappings = dict(zip(dimension['Variable Name'], dimension['Description']))
        # Rename columns in df using the mapping
        df.rename(columns=mappings,inplace=True)

        # Store year and quarter cols
        df['Year'] = int(filename[:4])
        df['Quarter'] = filename[5:7]
        
        # Append DataFrame to the list
        dfs_list.append(df)
        
# Concatenate all DataFrames in the list
final_df = pd.concat(dfs_list, ignore_index=True)

In [52]:
# Get the sector mapping in


,Country,Net profit/total assets,Total liabilities/total assets,Working capital/total assets,Current assets/short-term liabilities,Retained earnings/total assets,Gross profit/total assets,Book value of equity/total liabilities,Net sales revenue/total assets,Equity/total assets,...,EBIT (n)/EBIT (n−1),Net profit (n)/net profit (n−1),Inventory (n)/inventory (n−1),Receivables (n)/receivables (n−1),short-term liabilities (n)/short-term liabilities (n−1),Net cash flow from (used in) operating activities (n)/Net cash flow from (used in) operating activities (n−1),Net cash flow(n)/net cash flow (n−1),sectors,Year,Quarter
0,Hungary,m,m,m,m,m,m,m,m,m,...,m,m,m,m,m,m,m,1.0,2017,Q2
1,Poland,0,0.49,0.06,1.31,0.06,0.01,1.06,0.18,0.51,...,13.7,-14.88,1.11,1.12,1.04,m,0.59,1.0,2017,Q2
2,Hungary,0.01,0.78,-0.04,0.9,0,0.01,0.27,0.27,0.22,...,1.45,0.92,1.14,1.08,1.18,m,m,1.0,2017,Q2
3,Poland,-0.02,0.44,0.18,1.52,0.28,-0.01,1.29,0.21,0.56,...,0.74,0.89,1.6,0.6,0.48,m,0.73,1.0,2017,Q2
4,Poland,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,2017,Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7231,Slovakia,m,m,m,m,m,m,m,m,m,...,m,m,m,0,0,m,m,6.0,2021,Q1
7232,Poland,-0.15,29.05,-27.13,0.07,-36.58,-0.21,-0.96,0,-27.94,...,m,m,m,m,m,m,m,6.0,2021,Q1
7233,Slovakia,m,m,m,m,m,m,m,m,m,...,m,m,m,0,0,m,m,6.0,2021,Q1
7234,Poland,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.0,2021,Q1


In [53]:
sector

,code_sector,description_sector
0,1,Transportation and warehousing
1,2,Wholesale trade
2,3,Manufacturing
3,4,Retail trade
4,5,Energy
5,6,Construction


In [6]:
filename

'2017 Q4.arff'

In [8]:
with open(f'data/2017 Q4.arff', 'r') as f: # Open the .arff file
    raw_data = arff.load(f)

BadNumericalValue: Invalid numerical value, at line 533.

In [63]:
df

,Country,Net profit/total assets,Total liabilities/total assets,Working capital/total assets,Current assets/short-term liabilities,Retained earnings/total assets,Gross profit/total assets,Book value of equity/total liabilities,Net sales revenue/total assets,Equity/total assets,...,Total assets (n)/total assets (n−1),Current assets (n)/current assets (n−1),EBIT (n)/EBIT (n−1),Net profit (n)/net profit (n−1),Inventory (n)/inventory (n−1),Receivables (n)/receivables (n−1),short-term liabilities (n)/short-term liabilities (n−1),Net cash flow from (used in) operating activities (n)/Net cash flow from (used in) operating activities (n−1),Net cash flow(n)/net cash flow (n−1),sectors
0,Hungary,m,m,m,m,m,m,m,m,m,...,m,m,m,m,m,m,m,m,m,1.0
1,Poland,0,0.49,0.06,1.31,0.06,0.01,1.06,0.18,0.51,...,1.06,1.04,13.7,-14.88,1.11,1.12,1.04,m,0.59,1.0
2,Hungary,0.01,0.78,-0.04,0.9,0,0.01,0.27,0.27,0.22,...,1.15,1.19,1.45,0.92,1.14,1.08,1.18,m,m,1.0
3,Poland,-0.02,0.44,0.18,1.52,0.28,-0.01,1.29,0.21,0.56,...,0.73,0.57,0.74,0.89,1.6,0.6,0.48,m,0.73,1.0
4,Poland,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,Slovakia,m,m,m,m,m,m,m,m,m,...,m,m,m,m,m,m,m,m,m,6.0
446,Poland,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.0
447,Slovakia,m,m,m,m,m,m,m,m,m,...,m,m,m,m,m,m,m,m,m,6.0
448,Poland,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.0


In [ ]:
import pyodbc
import os

# Connection details
server = os.getenv('RELATIONALDB_HOST')
database = os.getenv('RELATIONALDB_DB')
username = os.getenv('MSSQL_USER')
password = os.getenv('SA_PASSWORD')

# Create connection string
conn_str = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Connect to SQL Server
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Execute SQL queries
cursor.execute('SELECT * FROM your_table')
rows = cursor.fetchall()

# Process query results
for row in rows:
    print(row)

# Close connection
conn.close()


In [ ]:
1+1